# Feature analysis and extraction

This notebooks contain a more detailed analysis of each column.
For each column of the table, we'll try to assess the type of distribution
(e.g. does it fit a Gaussian?).
Then we'll create as many features as possible accordingly
(e.g. one column could lend itself to both a numerical feature and
a binary or categorical feature).

### Read the data

In [13]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

%matplotlib inline

Read the data:

In [2]:
order_data = pd.read_json(
    "../data/order_data_dtypes.json",
    orient='table',
)

Both time-series and values distribution can be useful for building features so we'll focus on both.

In [3]:
order_data.head()

,order_datetime,customer_id,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,2015-06-20 19:00:00,000097eabfd9,1,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,2016-01-29 20:00:00,0000e2c6d9be,1,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,2017-02-26 19:00:00,000133bb597f,1,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324
3,2017-02-05 17:00:00,00018269939b,1,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356
4,2015-08-04 19:00:00,0001a00468a6,1,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356


In [4]:
order_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 786547 entries, 0 to 786546
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_datetime       786547 non-null  datetime64[ns]
 1   customer_id          786547 non-null  object        
 2   customer_order_rank  786547 non-null  int64         
 3   is_failed            786547 non-null  int64         
 4   voucher_amount       786547 non-null  float64       
 5   delivery_fee         786547 non-null  float64       
 6   amount_paid          786547 non-null  float64       
 7   restaurant_id        786547 non-null  category      
 8   city_id              786547 non-null  category      
 9   payment_id           786547 non-null  category      
 10  platform_id          786547 non-null  category      
 11  transmission_id      786547 non-null  category      
dtypes: category(5), datetime64[ns](1), float64(3), int64(2), object(1)
memor

### Orders features

Let's store our newly-created features in separate tables.

Let's create a table for order features, including one-hot representation of categorical variables (dummy variables):

In [5]:
# orders_features = pd.DataFrame(index=order_data.index)

# orders_features['customer_id'] = order_data['customer_id']

# orders_features.head()

### Customer features

We'll be extracting customer features and storing them in a separate dataframe:

In [6]:
# customer_features = pd.DataFrame(index=order_data['customer_id'].unique())
# customer_features.index.name = 'customer_id'

In [7]:
# customer_features

In [8]:
# orders_features['hour_of_day'] = order_data['order_datetime'].dt.hour

In [9]:
# orders_features['day_of_week'] = order_data['order_datetime'].dt.dayofweek

In [10]:
# orders_features['is_holiday'] = (
#     (
#         (order_data['order_datetime'].dt.month == 1) & (order_data['order_datetime'].dt.day == 1)
#     ) | (
#         (order_data['order_datetime'].dt.month == 12) & ((order_data['order_datetime'].dt.day == 25) | (order_data['order_datetime'].dt.day == 31))
#     )
# )

In [11]:
# orders_features.info()

#### Dummy variables

Let's produce dummy variables from these:

In [15]:
# col_name = 'hour_of_day'
# pd.get_dummies(
#     order_data['order_datetime'].dt.hour,
#     prefix=col_name,
# ).to_parquet(
#     f"../data/{col_name}_dummy.parquet",
#     index=order_data['customer_id'],
# )

In [21]:
col_name = 'hour_of_day'
pq.write_to_dataset(
    pa.Table.from_pandas(
        pd.get_dummies(
            order_data.set_index('customer_id')['order_datetime'].dt.hour,
            prefix=col_name,
        ).reset_index(),
    ),
    root_path=f"../data/parquet/{col_name}_dummy.parquet",
    partition_cols=['customer_id'],
)

KeyboardInterrupt: 

In [ ]:
pq.read_table("../data/parquet/hour_of_day_dummy.parquet/").head()

In [12]:
!ls ../data/

city_id_dummy.parquet				order_data.parquet
day_of_week_dummy.parquet			orders_table_dtypes.json
hour_of_day_dummy.parquet			orders_table_dtypes_log.json
machine_learning_challenge_labeled_data.csv.gz	payment_id_dummy.parquet
machine_learning_challenge_order_data.csv.gz	platform_id_dummy.parquet
order_data_batch.json				restaurant_id_dummy.parquet
order_data_dtypes.json				transmission_id_dummy.parquet
order_data_dtypes_log.json


In [13]:
# col_name = 'hour_of_day'
# hour_dummies = pd.get_dummies(
#     orders_features[col_name],
#     prefix=col_name,
# )

In [14]:
# orders_features[hour_dummies.columns] = hour_dummies

In [15]:
col_name = 'day_of_week'
day_of_week_dummies = pd.get_dummies(
    order_data['order_datetime'].dt.dayofweek,
    prefix=col_name,
).to_parquet(
    f"../data/{col_name}_dummy.parquet",
    index=order_data['customer_id'],
)

In [16]:
# orders_features[day_of_week_dummies.columns] = day_of_week_dummies

In [ ]:
# orders_features.info()

In [17]:
# last_orders = order_data.groupby('customer_id')['order_datetime'].max()
# max_datetime = order_data['order_datetime'].max()
# max_datetime

In [18]:
# customer_features['last_order_age_days'] = (max_datetime - last_orders).dt.days

In [19]:
# first_orders = order_data.groupby('customer_id')['order_datetime'].min()

In [20]:
# customer_features['first_order_age_days'] = (max_datetime - first_orders).dt.days

In [21]:
# customer_features.head()

### Number of orders

We'll throw in the number of orders per customer as well:

In [22]:
# number_of_orders_per_customer = order_data.groupby('customer_id')['amount_paid'].count()
# number_of_orders_per_customer.name = 'n_orders'

In [23]:
# customer_features = customer_features.join(number_of_orders_per_customer)
# customer_features.head()

### Customer order rank

The customer order rank can show us how many orders a customer has
successfully submitted in the considered time frame.
We find this by taking the maximum of the quantity.

In [24]:
# max_customer_order_rank = order_data.groupby(
#     'customer_id'
# )['customer_order_rank'].max().sort_values(ascending=False)
# max_customer_order_rank

In [25]:
# max_customer_order_rank.describe()

In [26]:
# max_customer_order_rank.quantile(.99)

As we can see, 99% or customers haven't ordered more than 31 times,
and the majority haven't ordered more than one.

In [27]:
# max_customer_order_rank.name = 'max_customer_order_rank'
# customer_features = customer_features.join(max_customer_order_rank)
# customer_features.info()

### is_failed

Here we can see how many orders have failed.
Using the number of orders we can also calculate a rate of success.

In [28]:
# failed_orders_per_customer = order_data.groupby('customer_id')['is_failed'].sum()
# failed_orders_per_customer.name = 'n_failed'

In [29]:
# failed_orders_per_customer.head()

In [30]:
# customer_features = customer_features.join(failed_orders_per_customer)
# customer_features.info()

### Voucher amount

We can extract the number of vouchers used, the total amount from vouchers,
and the maximum value of a customer's voucher.

In [31]:
# max_voucher_amount = order_data.groupby('customer_id')['voucher_amount'].max()
# max_voucher_amount.name = 'max_voucher_amount'

In [32]:
# max_voucher_amount.describe()

In [33]:
# max_voucher_amount.quantile(.9999)

In [34]:
# customer_features = customer_features.join(max_voucher_amount)
# customer_features.info()

In [35]:
# tot_voucher_amount = order_data.groupby('customer_id')['voucher_amount'].sum()
# tot_voucher_amount.name = 'tot_voucher_amount'

In [36]:
# tot_voucher_amount.describe()

In [37]:
# customer_features = customer_features.join(tot_voucher_amount)
# customer_features.info()

In [38]:
# n_vouchers = (order_data['voucher_amount']>0).groupby(order_data['customer_id']).sum()
# n_vouchers.name = 'n_vouchers'

In [39]:
# n_vouchers.describe()

In [40]:
# customer_features = customer_features.join(n_vouchers)

In [41]:
# customer_features.info()

### Delivery fee

here we follow an identical procedure as for vouchers.

In [42]:
# tot_delivery_fee = order_data.groupby('customer_id')['delivery_fee'].sum()
# max_delivery_fee = order_data.groupby('customer_id')['delivery_fee'].max()
# # How many times a delivery fee was paid
# n_delivery_fee = (order_data['delivery_fee']>0).groupby(order_data['customer_id']).sum()

In [43]:
# tot_delivery_fee.name = 'tot_delivery_fee'
# max_delivery_fee.name = 'max_delivery_fee'
# n_delivery_fee.name = 'n_delivery_fee'

In [44]:
# customer_features = customer_features.join(tot_delivery_fee)
# customer_features = customer_features.join(max_delivery_fee)
# customer_features = customer_features.join(n_delivery_fee)
# customer_features.info()

### Amount paid

Here we can use the average amount paid on top of everything else.

In [45]:
# tot_amount_paid = order_data.groupby('customer_id')['amount_paid'].sum()
# avg_amount_paid = order_data.groupby('customer_id')['amount_paid'].mean()
# max_amount_paid = order_data.groupby('customer_id')['amount_paid'].max()
# min_amount_paid = order_data.groupby('customer_id')['amount_paid'].min()

In [46]:
# tot_amount_paid.name = 'tot_amount_paid'
# avg_amount_paid.name = 'avg_amount_paid'
# max_amount_paid.name = 'max_amount_paid'
# min_amount_paid.name = 'min_amount_paid'

In [47]:
# customer_features = customer_features.join(tot_amount_paid)
# customer_features = customer_features.join(avg_amount_paid)
# customer_features = customer_features.join(max_amount_paid)
# customer_features = customer_features.join(min_amount_paid)

In [48]:
# customer_features.info()

### Restaurant ID

Categorical variables lend themselves naturally to produce dummy variables.

In [ ]:
# n_restaurants = order_data.groupby('customer_id')['restaurant_id'].nunique()
# n_restaurants.name = 'n_restaurants'
# n_restaurants

In [49]:
# n_restaurants.describe()

In [50]:
# customer_features = customer_features.join(n_restaurants)
# customer_features.info()

#### Dummy variables

In [ ]:
# col_name = 'restaurant_id'
# rest_id_dummies = pd.get_dummies(
#     order_data[col_name],
#     prefix=col_name,
#     sparse=True,
# )

In [51]:
col_name = 'restaurant_id'
# rest_id_dummies = 
pd.get_dummies(
    order_data[col_name],
    prefix=col_name,
#     sparse=True,
).to_parquet(
    f"../data/{col_name}_dummy.parquet",
    index=order_data['customer_id'],
)

In [52]:
# rest_id_dummies.info()

In [53]:
# orders_features[rest_id_dummies.columns] = rest_id_dummies

In [54]:
# orders_features.info()

In [56]:
# orders_features.head()

### City ID

In [57]:
# n_cities = order_data.groupby('customer_id')['city_id'].nunique()
# n_cities.name = 'n_cities'
# n_cities

In [58]:
# n_cities.describe()

In [59]:
# customer_features = customer_features.join(n_cities)

In [60]:
# customer_features.info()

#### Dummy vars

In [61]:
col_name = 'city_id'
# city_id_dummies = 
pd.get_dummies(
    order_data[col_name],
    prefix=col_name,
#     sparse=True,
).to_parquet(
    f"../data/{col_name}_dummy.parquet",
    index=order_data['customer_id'],
)

In [62]:
# city_id_dummies.info()

In [63]:
# orders_features[city_id_dummies.columns] = city_id_dummies

In [64]:
# orders_features.info()

### Payment ID

In [65]:
col_name = 'payment_id'
# payment_id_dummies = 
pd.get_dummies(
    order_data[col_name],
    prefix=col_name,
#     sparse=True,
).to_parquet(
    f"../data/{col_name}_dummy.parquet",
    index=order_data['customer_id'],
)

In [66]:
# payment_id_dummies.info()

In [67]:
# orders_features[payment_id_dummies.columns] = payment_id_dummies

In [68]:
# orders_features.info()

### Platform ID

In [69]:
col_name = 'platform_id'
# platform_id_dummies = 
pd.get_dummies(
    order_data[col_name],
    prefix=col_name,
#     sparse=True,
).to_parquet(
    f"../data/{col_name}_dummy.parquet",
    index=order_data['customer_id'],
)

In [70]:
# platform_id_dummies.info()

In [71]:
# orders_features[platform_id_dummies.columns] = platform_id_dummies

In [72]:
# orders_features.info()

### Transmission ID

In [73]:
col_name = 'transmission_id'
# transmission_id_dummies = 
pd.get_dummies(
    order_data[col_name],
    prefix=col_name,
#     sparse=True,
).to_parquet(
    f"../data/{col_name}_dummy.parquet",
    index=order_data['customer_id'],
)

In [74]:
# transmission_id_dummies.info()

In [75]:
# orders_features[transmission_id_dummies.columns] = transmission_id_dummies

In [76]:
# orders_features.info()

### Store data

This time a space-efficient format is preferrable.

In [77]:
# customer_features

### Customer features from dummy features

Here we distill the extracted order feature to customer features.

In [78]:
# orders_features

In [79]:
# orders_features.info()

In [ ]:
# customer_features_dummy = orders_features.drop(
#     columns=['hour_of_day', 'day_of_week']
# ).groupby(['customer_id']).agg(np.sum)